In [1]:
import numpy as np
import mygrad as mg
from mygrad import Tensor

from noggin import create_plot
import matplotlib.pyplot as plt

%matplotlib inline

In [10]:
import re, string

class Database:
    def get_IDF(word):
        return np.log10( (len(coco_data["annotations"])) / len(img2captions[word]))
    
    def get_gloveE(caption): 
        return glove[caption]
    
    def get_word_vectors(line):
        strip_punc(line)
        line = line.lower()
        arr = line.split()
        verctor_sum = 0
        for word in arr:
            verctor_sum += (get_IDF(word) * get_glovE(word))
        return verctor_sum


In [ ]:
#Dictionary 3:
#caption-ID -> caption (e.g. 24 -> "two dogs on the grass")

capID2cap = {}

for caption in coco_data["annotations"]:
    capID2cap[caption["id"]] = caption["caption"]

In [ ]:
#Dictionary 2:
#caption-ID -> image-ID

cap2img = {}

for caption in coco_data["annotations"]:
    cap2img[caption["id"]] = caption["image_id"]

In [ ]:
#Dictionary 1:
#image-ID -> [cap-ID-1, cap-ID-2, ...]

img2captions = {}

for cap in coco_data["annotations"]:
    if cap["image_id"] not in img2captions:
        img2captions[cap["image_id"]] = []
    img2captions[cap["image_id"]].append(cap["id"])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def datasplit(X):
  X_train, X_test = train_test_split(X, train_size=0.8, random_state=50)
  return (X_train, X_test)

In [ ]:

#done!
def generateTriples(imglist, img2captions):
  ret = []
  for img in imglist:
    for caption in img2captions[img]:
      fake = np.random.choice(imglist)
      while (fake == img):
        fake = np.random.choice(imglist)
      ret.append((caption, img, fake))
  return ret




In [ ]:

img2captions = {1: [100, 200, 300], 2: [1,2,3], 3:[4,5], 4:[6]}
imglist = [1,2,3,4]
train, test = datasplit(imglist)
hehe = generateTriples(train, img2captions)
print(hehe)

In [ ]:
from mynn.layers.dense import dense
from mygrad.nnet.initializers import glorot_uniform

In [ ]:
class Model:

    def __init__(self, D_input=512, D_glove=200):
        self.dense1 = dense(D_input, D_glove, weight_initializer=glorot_normal) 

    def __call__(self, x):
        return self.dense1(x)

    @property
    def parameters(self):
        return self.dense1.parameters

In [ ]:
from mynn.optimizers.sgd import SGD
from mygrad.nnet.losses import margin_ranking_loss

optim = SGD(model.parameters, learning_rate=1e-3, momentum=0.9)

In [1]:
# (1,200)       (1,512)          (1,512)

# [caption1, confusion_image, truth_image]
# |caption2, confusion_image, truth_image|
# [caption3, confusion_image, truth_image]

SyntaxError: invalid syntax (4090296465.py, line 2)

In [ ]:
def accuracy(prediction_truth, prediction_confusion, truth_caption):
    truth_count = np.count_nonzero(np.dot(prediction_truth, truth_caption) > np.dot(prediction_confusion, truth_caption))
    return truth_count / len(truth_caption)

In [ ]:
batch_size = 32
num_epochs = 1

model = Model(512, 200)

for epoch_cnt in range(num_epochs):
    idxs = np.arange(len(train)) 
    np.random.shuffle(idxs)

    for batch_cnt in range(len(train) // batch_size):
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size]
        
        # captions
        truth_caption_ID = train[batch_indices][0]
        truth_caption = truth_caption[truth_caption_ID]
        #image and confusion
        truth_image = train[batch_indices][1]
        confusion = train[batch_indices][2]
        
        prediction_truth = model(truth_image)  
        prediction_confusion = model(confusion)
        
        sim_truth = np.dot(prediction_truth, truth_caption)
        sim_confusion =  np.dot(prediction_confusion, truth_caption)
        
        # add einsum 
        loss = margin_ranking_loss(sim_truth, sim_confusion, truth_caption, margin=0.25) 

        loss.backward() 

        optim.step() 

        acc = accuracy(prediction_truth, prediction_confusion, truth_caption)
        
        plotter.set_train_batch(
            {"loss": loss.item(), "accuracy": acc}, batch_size=batch_size
        )
        
    for batch_cnt in range(0, len(x_test) // batch_size):
        idxs = np.arange(len(x_test))
        batch_indices = idxs[batch_cnt * batch_size : (batch_cnt + 1) * batch_size] 
        batch = x_test[batch_indices]

        with mg.no_autodiff:
            # captions
            truth_caption_ID = test[batch_indices][0]
            truth_caption = truth_caption[truth_caption_ID]
            #image and confusion
            truth_image = test[batch_indices][1]
            confusion = test[batch_indices][2]

            prediction_truth = model(truth_image)  
            prediction_confusion = model(confusion)
            
            acc = accuracy(prediction_truth, prediction_confusion, truth_caption)

        plotter.set_test_batch({"accuracy": acc}, batch_size=batch_size)

    plotter.set_train_epoch()
    plotter.set_test_epoch()
plotter.plot()

In [ ]:
ids = np.array(IDs)
# only fill w validation set images to know that the quality of our results isn't overfitting on our data
imageVectors = np.array(vectors)

def query(captionVector):
    dot = np.dot(imageVectors, captionVector)
    return ids[np.argpartition(dot, -4)[-4:]]

In [ ]:
import io

import requests
from PIL import Image

def download_image(img_url: str) -> Image:
    response = requests.get(img_url)
    return Image.open(io.BytesIO(response.content))

In [ ]:
from matplotlib import pyplot as plt
  
# create figure
fig = plt.figure(figsize=(10, 7))
  
# setting values to rows and column variables
rows = 2
columns = 2
  
images = query(caption_vector)
# reading images
Image1 = coca_data[images[0]].url
Image2 = coca_data[images[1]].url
Image3 = coca_data[images[2]].url
Image4 = coca_data[images[3]].url
  
# Adds a subplot at the 1st position
fig.add_subplot(rows, columns, 1)
  
# showing image
plt.imshow(Image1)
plt.axis('off')
plt.title("First")
  
# Adds a subplot at the 2nd position
fig.add_subplot(rows, columns, 2)
  
# showing image
plt.imshow(Image2)
plt.axis('off')
plt.title("Second")
  
# Adds a subplot at the 3rd position
fig.add_subplot(rows, columns, 3)
  
# showing image
plt.imshow(Image3)
plt.axis('off')
plt.title("Third")
  
# Adds a subplot at the 4th position
fig.add_subplot(rows, columns, 4)
  
# showing image
plt.imshow(Image4)
plt.axis('off')
plt.title("Fourth")